In [6]:
import pandas as pd

# Specify the output JSON file path
json_file_path = r"D:\FYP\Final Model Training - Copy\GPU\converted_output_file.json"

# Read the JSON file into a pandas DataFrame
imdb = pd.read_json(json_file_path)

# Display the DataFrame
# print(df)


In [7]:
df['test'][0]

{'text': 'bBelgian cardinals prevent priests to be in a private room with children.',
 'label': 0}

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=310 ,truncation=True)

In [12]:
tokenized_imdb = imdb["test"].map(preprocess_function)

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
import evaluate

accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

KeyError: 'train'

In [29]:
import csv
import json

# Specify the input CSV file path
csv_file_path = r"D:\FYP\Final Model Training\Data Set\Final_data_set_shuffle_for_model.csv"

# Specify the output JSON file path
json_file_path = 'converted_output_file.json'

# Initialize an empty dictionary to store the converted data
converted_data = {"test": []}

# Open the CSV file in read mode
with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
    # Create a CSV reader object
    reader = csv.DictReader(csvfile)
    
    # Iterate over each row in the CSV file
    for row in reader:
        # Create a dictionary in the desired format
        data_dict = {
            'text': row['Headline'],
            'label': int(row['Label'])
        }
        
        # Append the dictionary to the list inside the 'test' key
        converted_data["test"].append(data_dict)

# Write the converted data to a JSON file
with open(json_file_path, 'w', encoding='utf-8') as jsonfile:
    # Write the data as JSON
    json.dump(converted_data, jsonfile, indent=4)
